In [2]:
# This algorithm creates Look Up Table (LUT) by simulating different microphysical variables. 
# This algorithm will automatically create new output directory and store data according to --
# the ranges of microphysical variable as you change the constant microphysical variable
# The line needs to be changed per simulation are remarked by a comment on the right side

#-----------------code start----------------#

import csv
import numpy as np
import os
import shutil

def Wcloud(variable_const,variable_iter,CTH,CBH):
    inp = open("UVSPEC_REPTRAN_THERMAL.INP","w+") 
    inp.write("data_files_path /opt/libradtran/2.0.2/share/libRadtran/data\r\n")
    inp.write("atmosphere_file /home/sshohan/libRadtran/AFGLUS.70KM\r\n")
    inp.write("rte_solver disort\r\n")
    inp.write("pseudospherical\r\n")
    inp.write("mol_abs_param reptran fine\r\n")
    inp.write("output_quantity brightness\r\n")
    inp.write("radiosonde /home/sshohan/libRadtran/dropsonde_D20200124_134504_PQC.txt H2O RH\r\n")
    inp.write("source thermal\r\n")
    inp.write("wavelength 7700 12000\r\n")
    inp.write("time 2020 01 24 13 45 04")
    inp.write("latitude N 14 22 835")
    inp.write("longitude W 58 09 833")
    inp.write("zout 10.33\r\n") #altitude
    inp.write("filter_function_file /home/sshohan/libRadtran/Filter_function.txt normalize\r\n")
    inp.write("wc_properties mie interpolate\r\n")
    inp.write("wc_file 1D  /home/sshohan/libRadtran_LUT/wc_mp.dat\r\n")
    inp.write("umu 1.0\r\n") #there would be a loop for later simulation of flight azimuth angle
    inp.write("output_process integrate\r\n")
    inp.write("output_user lambda uu\r\n")
    inp.close()
    for i in variable_const:    
        for j in variable_iter:
            with open("wc_mp.dat","w+") as file:
                csv_writer = csv.writer(file, delimiter=" ")
                csv_writer.writerow(['#alt(km)','LWC(g/m3)','r_eff(um)'])
                csv_writer.writerow([CTH,0,0])
                csv_writer.writerow([CBH,j,i]) ###ONLY CHANGE### switch order between(i and j) according to variables_const and variables_iter
            os.popen('uvspec <UVSPEC_REPTRAN_THERMAL.INP> TEST_REPTRAN_THERMAL.out').read()
            try:
                dirName = 'output_'+str(i) 
                os.mkdir(dirName)
            except:
                OSError
            dst_path = '/home/sshohan/libRadtran_LUT/'+str(dirName)+'/outfile_'+str(j)+'.txt' 
            origin_path = './TEST_REPTRAN_THERMAL.out'
            not os.path.exists(dst_path) or os.makedirs(dst_path)
            shutil.copy(origin_path, dst_path) 
            data = np.genfromtxt(dirName)
            with open("LUT_"+str(i)+".txt","w+") as wfile:
                csv_writer = csv.writer(wfile, delimiter= " ")
                for k in range(0, len(variable_iter)):
                    csv_writer.writerow([str(variable_iter[k]),str(data[1])])
            
variable_const = np.arange(4,5) ## select variable which will be constant for each iteration 
variable_iter = np.linspace(0.1,0.3,num=2).round(decimals=2) ##select variable for iteration 
CTH = 1 ## cloud top height
CBH = 0.450 ## cloud base height
Wcloud(variable_const,variable_iter,CTH,CBH)

#-----------------code end----------------#

#variables_const = np.arange(4,6)
#LWC=np.linspace(0.1,1,num=3).round(decimals=2)
#r_eff = np.linspace(4,15,num=10).round(decimals=2)
#Wcloud(LWC)



In [21]:
data = np.genfromtxt('TEST_REPTRAN_THERMAL.out')
data[1]

288.9849243

In [ ]:
with open("TEST_REPTRAN_THERMAL.out", "r"), open('xyz.txt', 'w') as textfile, myfile:
    for line in textfile:
        var1, var2 = line.split(",");
        myfile.writelines(var1)

In [24]:
np.arange(4,6,1)

array([4, 5])

In [33]:
import numpy as np
from glob import glob
matrices = []
def read_all_files():
    file_names = glob('output_4/output_*.txt')
    arrays = [np.loadtxt(f) for f in file_names]
    matrices = np.concatenate(arrays)
    print(matrices)

In [32]:
matrices

[]

In [37]:

import os

# create a dictionary with file names as keys
# and for each file name the paths where they
# were found
file_paths = {'output_}
for root, dirs, files in os.walk('.'):
    for f in files:
        if f.endswith('.txt'):
            if f not in file_paths:
                file_paths[f] = []
            file_paths[f].append(root)


NameError: name 'output_' is not defined